## Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Process data

We want to define :
- Definition of  default
- Time step (12 months, 24 months)

If the lender default after 24 months we do not take it into account.


We define the default as having 90 days of non payment or having zero_balance code a 02, 03 or 09 (cf data details) between the "FIRST PAYMENT DATE" and the "MONTLHLY REPORTING PERIOD" being under 12 months.

In [ ]:
##### make_labels.py initial code

# Paths to origination and performance datasets
path_orig = "../data/raw/mortgage_data/historical_data_2022Q1/historical_data_2022Q1.txt"
path_perf = "../data/raw/mortgage_data/historical_data_2022Q1/historical_data_time_2022Q1.txt"

# Column names for the Origination dataset
colnames_origination = [
    "credit_score",                         # Borrower credit score
    "first_payment_date",                   # First scheduled payment date (YYYYMM)
    "first_time_homebuyer_flag",            # First-time homebuyer flag
    "maturity_date",                        # Loan maturity date (YYYYMM)
    "msa_md",                               # MSA / Metropolitan Division
    "mi_percent",                           # Mortgage Insurance percentage
    "number_of_units",                      # Number of units in the property
    "occupancy_status",                     # Occupancy status (owner/second/investment)
    "original_cltv",                        # Original Combined Loan-to-Value
    "original_dti",                         # Original Debt-to-Income ratio
    "original_upb",                         # Original Unpaid Principal Balance
    "original_ltv",                         # Original Loan-to-Value
    "original_interest_rate",               # Original Interest Rate
    "channel",                              # Origination channel (Retail, Broker, etc.)
    "ppm_flag",                             # Prepayment penalty flag
    "amortization_type",                    # Amortization type (FRM/ARM)
    "property_state",                       # Property state (2-letter code)
    "property_type",                        # Property type
    "postal_code",                          # Postal code (last 2 digits = 00)
    "loan_sequence_number",                 # Unique loan identifier (primary key)
    "loan_purpose",                         # Loan purpose (Purchase, Refinance)
    "original_loan_term",                   # Original loan term (months)
    "number_of_borrowers",                  # Number of borrowers
    "seller_name",                          # Seller name
    "servicer_name",                        # Servicer name
    "super_conforming_flag",                # Super conforming flag
    "pre_relief_refi_loan_seq_number",      # Pre-relief refinance loan sequence number
    "special_eligibility_program",          # Special eligibility program
    "relief_refinance_indicator",           # Relief refinance indicator
    "property_valuation_method",            # Property valuation method
    "interest_only_indicator",              # Interest-only indicator
    "mi_cancellation_indicator"             # MI cancellation indicator
]

# Column names for the Performance dataset
colnames_performance = [
    "loan_sequence_number",                 # Loan identifier (primary key)
    "monthly_reporting_period",             # Reporting period (YYYYMM)
    "current_actual_upb",                   # Current actual Unpaid Principal Balance
    "current_loan_delinquency_status",      # Loan delinquency status (0,1,2,..., RA)
    "loan_age",                             # Loan age in months
    "remaining_months_to_legal_maturity",   # Remaining months until legal maturity
    "defect_settlement_date",               # Defect settlement date (YYYYMM)
    "modification_flag",                    # Loan modification flag
    "zero_balance_code",                    # Zero balance code (01,02,03,09, etc.)
    "zero_balance_effective_date",          # Zero balance effective date (YYYYMM)
    "current_interest_rate",                # Current interest rate
    "current_non_interest_bearing_upb",     # Current non-interest-bearing UPB
    "ddlpi",                                # Due Date of Last Paid Installment (YYYYMM)
    "mi_recoveries",                        # Mortgage insurance recoveries
    "net_sale_proceeds",                    # Net sale proceeds
    "non_mi_recoveries",                    # Non-MI recoveries
    "total_expenses",                       # Total expenses
    "legal_costs",                          # Legal costs
    "maintenance_and_preservation_costs",   # Maintenance & preservation costs
    "taxes_and_insurance",                  # Taxes and insurance
    "miscellaneous_expenses",               # Miscellaneous expenses
    "actual_loss_calculation",              # Actual loss calculation
    "cumulative_modification_cost",         # Cumulative modification cost
    "step_modification_flag",               # Step modification flag
    "payment_deferral",                     # Payment deferral indicator
    "estimated_ltv",                        # Estimated Loan-to-Value (ELTV)
    "zero_balance_removal_upb",             # Zero balance removal UPB
    "delinquent_accrued_interest",          # Delinquent accrued interest
    "delinquency_due_to_disaster",          # Delinquency due to disaster
    "borrower_assistance_status_code",      # Borrower assistance status code
    "current_month_modification_cost",      # Current month modification cost
    "interest_bearing_upb"                  # Interest-bearing UPB
]

# Load both datasets (pipe-delimited, no header)
df_train_train_orig = pd.read_csv(path_orig, sep="|", header=None, names=colnames_origination)
df_perf = pd.read_csv(path_perf, sep="|", header=None, names=colnames_performance)


# Shape data
print(f"Shape data orig : {df_orig.shape}")
print(f"Shape data perf: {df_perf.shape}")

# Convert YYYYMM to datetime
df_perf["monthly_reporting_period"] = pd.to_datetime(
    df_perf["monthly_reporting_period"].astype(str), 
    format="%Y%m"
)
df_orig["first_payment_date"] = pd.to_datetime(
    df_orig["first_payment_date"].astype(str), 
    format="%Y%m"
)

# Convert to monthly Period type (YYYY-MM), easier for month arithmetic
df_perf["monthly_reporting_period"] = df_perf["monthly_reporting_period"].dt.to_period("M")
df_orig["first_payment_date"] = df_orig["first_payment_date"].dt.to_period("M")

# Merge Origination info into Performance dataset
df_perf = pd.merge(
    df_perf, 
    df_orig[["loan_sequence_number", "first_payment_date"]], 
    on="loan_sequence_number", 
    how="left"
)

# Compute loan age in months since origination
df_perf["months_since_orig"] = (
    df_perf["monthly_reporting_period"] - df_perf["first_payment_date"]
).apply(lambda x: x.n)

# Flag observations within the first 24 months after origination
df_perf["within_24m"] = df_perf["months_since_orig"] <= 24



df_perf_within = df_perf.copy()
# print(df_perf_within.shape)
df_perf_within = df_perf_within[df_perf_within["within_24m"] == True]
# print(df_perf_within.shape)




# Define default:
# - delinquency status not in {0,1,2} (=> 90+ days delinquent or RA)
# - OR zero_balance_code in {03,09,15,16,96} (dispositions / foreclosures)
df_perf_within["default"] = np.where(
    (~df_perf_within["current_loan_delinquency_status"].astype(str).isin(["0","1","2"])) |
    (df_perf_within["zero_balance_code"].astype(str).isin(["03","09","15","16","96"])),
    1, 0
)

# Aggregate at loan level: max(default) = 1 if loan ever defaulted in 24m
loan_level = (
    df_perf_within.groupby("loan_sequence_number")["default"]
    .max()  # 0 if always current, 1 if ever default
    .reset_index()
)

df_orig = pd.merge(df_orig, loan_level, on = "loan_sequence_number", how = "left")

# Feature engeenering


When building a credit scoring model, it’s essential to respect the time dimension: the model should be trained on past vintages of loans and evaluated on future vintages. For example, if we have data for 2021Q4, 2022Q1, and 2022Q2, you would train on the first two quarters and keep 2022Q2 strictly for testing. This “out-of-time” validation mimics the real-world situation where a model is always used to predict the future, and it ensures that performance and calibration are not artificially inflated by information leakage across time.

In [ ]:
from pathlib import Path
import re
import pandas as pd

PATH = Path("../data/processed/default_labels")
PATTERN = re.compile(r"default_labels_24m_(\d{4}Q[1-4])\.csv$", re.I)

# Optionnel: accélère et économise de la RAM (pandas ≥ 2.0)
READ_KW = dict(engine="pyarrow", dtype_backend="pyarrow")
# Optionnel: ne charger que les colonnes utiles
# READ_KW["usecols"] = ["loan_sequence_number", "default_24m", "vintage", ...]

files = sorted(
    (p for p in PATH.glob("default_labels_24m_*.csv") if PATTERN.match(p.name)),
    key=lambda p: PATTERN.match(p.name).group(1)
)

buckets = {"train": [], "validation": [], "test": []}
def year_to_split(y: int):
    if 2021 <= y <= 2022: return "train"
    if y == 2023:         return "validation"
    if y == 2024:         return "test"
    return None  # ignore/alerter si autre année

for p in files:
    qstr = PATTERN.match(p.name).group(1)       # ex: "2021Q3"
    q = pd.Period(qstr, freq="Q")
    split = year_to_split(q.year)
    if split is None:
        print(f"Ignoré: {p.name}")
        continue
    df = pd.read_csv(p, **READ_KW)
    df["vintage"] = q                            # utile pour groupby/tri
    buckets[split].append(df)

# Concat une seule fois par split (plus rapide que concat successifs)
df_train       = pd.concat(buckets["train"], ignore_index=True)
df_validation  = pd.concat(buckets["validation"], ignore_index=True)
df_test        = pd.concat(buckets["test"], ignore_index=True)

# (Optionnel) dédup si nécessaire
# for d in (df_train, df_validation, df_test):
#     d.drop_duplicates(subset="loan_sequence_number", inplace=True)

# Proportions jolies
sizes = {"train": len(df_train), "validation": len(df_validation), "test": len(df_test)}
total = sum(sizes.values())
for k, n in sizes.items():
    print(f"{k.capitalize():<12}: {n:>10,} rows  ({n/total:.2%})")

In [ ]:
df_train.dtypes

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype

df_train_clean = df_train.copy()

# --- Helpers ---
def yn_space_to_bool(s, na_vals=('9',)):
    s = s.astype('string').str.strip().str.upper()
    # blank (espace/vide) = False par défaut pour certains indicateurs (cf. doc)
    s = s.fillna('').replace({' ': ''})
    out = s.map({'Y': True, 'N': False, '': False})
    out[s.isin(na_vals)] = pd.NA
    return out.astype('boolean')

def to_periodM(col):
    s = col.astype('string').str.strip()
    s = s.where(s.str.fullmatch(r'\d{6}'), pd.NA)   # YYYYMM
    return pd.PeriodIndex(s, freq='M')

def to_periodQ(col):
    s = col.astype('string').str.strip().str.upper()
    # suppose un format type '2016Q4'
    return pd.PeriodIndex(s.where(s.str.fullmatch(r'\d{4}Q[1-4]'), pd.NA), freq='Q')

# --- Dates / périodes ---
df_train_clean['first_payment_date'] = pd.to_datetime(df_train_clean['first_payment_date'],
                                                       errors="coerce").dt.to_period("M")
df_train_clean['maturity_date']      = to_periodM(df_train_clean['maturity_date'])
df_train_clean['vintage']            = to_periodQ(df_train_clean['vintage'])

# --- Booléens (selon doc) ---
df_train_clean['ppm_flag']                    = yn_space_to_bool(df_train_clean['ppm_flag'], na_vals=())
df_train_clean['interest_only_indicator']     = yn_space_to_bool(df_train_clean['interest_only_indicator'], na_vals=())
df_train_clean['super_conforming_flag']       = yn_space_to_bool(df_train_clean['super_conforming_flag'], na_vals=())
df_train_clean['first_time_homebuyer_flag']   = yn_space_to_bool(df_train_clean['first_time_homebuyer_flag'], na_vals=('9',))
# Relief refinance: Y / (blank)
df_train_clean['relief_refinance_indicator']  = yn_space_to_bool(df_train_clean['relief_refinance_indicator'])

# Si ta cible est 0/1 :
if pd.api.types.is_integer_dtype(df_train_clean['default_24m']) or pd.api.types.is_bool_dtype(df_train_clean['default_24m']):
    df_train_clean['default_24m'] = df_train_clean['default_24m'].map({1: True, 0: False}).astype('boolean')

# --- Sentinelles -> NA puis cast num ---
df_train_clean['credit_score']       = df_train_clean['credit_score'].replace({9999: pd.NA}).astype('Int16')
df_train_clean['mi_percent']         = df_train_clean['mi_percent'].replace({999: pd.NA}).astype('Int16')
df_train_clean['number_of_units']    = df_train_clean['number_of_units'].replace({99: pd.NA}).astype('Int8')
df_train_clean['original_cltv']      = df_train_clean['original_cltv'].replace({999: pd.NA}).astype('Int16')
df_train_clean['original_dti']       = df_train_clean['original_dti'].replace({999: pd.NA}).astype('Int16')
df_train_clean['original_ltv']       = df_train_clean['original_ltv'].replace({999: pd.NA}).astype('Int16')
df_train_clean['original_loan_term'] = df_train_clean['original_loan_term'].astype('Int16')
df_train_clean['number_of_borrowers']= df_train_clean['number_of_borrowers'].replace({99: pd.NA}).astype('Int8')
df_train_clean['original_upb']       = df_train_clean['original_upb'].astype('Int64')
df_train_clean['msa_md']             = df_train_clean['msa_md'].astype('Int32', errors='ignore')  # si NA déjà ok

# --- Identifiants / codes ---
df_train_clean['loan_sequence_number']          = df_train_clean['loan_sequence_number'].astype('string')
df_train_clean['pre_relief_refi_loan_seq_number']= df_train_clean['pre_relief_refi_loan_seq_number'].astype('string')

# Postal code : string + padding
df_train_clean['postal_code'] = (df_train_clean['postal_code']
    .astype('Int64')        # si c'était numérique
    .astype('string')
    .str.strip().str.upper()
    .str.zfill(5)
)

# --- Catégorielles (avec gestion des codes "NA") ---
df_train_clean['occupancy_status'] = (df_train_clean['occupancy_status'].astype('string').str.strip().str.upper()
                          .replace({'9': pd.NA})
                          .astype(CategoricalDtype(categories=['P','S','I'], ordered=False)))

df_train_clean['channel'] = (df_train_clean['channel'].astype('string').str.strip().str.upper()
                 .replace({'9': pd.NA})
                 .astype(CategoricalDtype(categories=['R','B','C','T'], ordered=False)))

df_train_clean['amortization_type'] = (df_train_clean['amortization_type'].astype('string').str.strip().str.upper()
                           .astype(CategoricalDtype(categories=['FRM','ARM'], ordered=False)))

df_train_clean['property_state'] = df_train_clean['property_state'].astype('string').str.strip().str.upper().astype('category')

df_train_clean['property_type'] = (df_train_clean['property_type'].astype('string').str.strip().str.upper()
                       .replace({'99': pd.NA})
                       .astype(CategoricalDtype(categories=['SF','CO','PU','CP','MH'], ordered=False)))

df_train_clean['loan_purpose'] = (df_train_clean['loan_purpose'].astype('string').str.strip().str.upper()
                      .replace({'9': pd.NA})
                      .astype(CategoricalDtype(categories=['P','C','N','R'], ordered=False)))

df_train_clean['special_eligibility_program'] = (df_train_clean['special_eligibility_program'].astype('string').str.strip().str.upper()
                                     .replace({'9': pd.NA})
                                     .astype(CategoricalDtype(categories=['H','F','R'], ordered=False)))

# Valeurs 1..4..9 => catégorie (pas ordonnée)
df_train_clean['property_valuation_method'] = (df_train_clean['property_valuation_method']
                                   .replace({9: pd.NA})
                                   .astype('Int8')
                                   .astype('category'))

In [ ]:
df_train_clean

In [ ]:
df_train_clean.head()

# Graph, data viz 

In [ ]:
counts = df_train["default_24m"].value_counts(normalize=True, dropna=False) * 100

# 4) Tracé Matplotlib (et non seaborn)
fig, ax = plt.subplots()
bars = ax.bar(counts.index.astype(str), counts.values, color="lightblue")

ax.set_title("Default modality proportion")
ax.set_xlabel("Modalité (default_24m)")
ax.set_ylabel("Pourcentage (%)")

# Légendes de pourcentage sur chaque barre
ax.bar_label(bars, labels=[f"{v:.1f}%" for v in counts.values], padding=3)

# Marges et rendu propre
ax.set_ylim(0, max(100, counts.max() * 1.15))   # laisse un peu d'espace pour le label
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Données (sans NaN)
x = df_train["original_upb"].dropna().values

# Bins robustes (Freedman–Diaconis) ; repli sur 'auto' si nécessaire
try:
    bins = np.histogram_bin_edges(x, bins='fd')
    if np.isinf(bins).any() or len(bins) < 2:
        raise ValueError
except Exception:
    bins = 'auto'

# Mise en forme "milliers avec espace"
fmt = FuncFormatter(lambda v, pos: f"{v:,.0f}".replace(",", " "))

fig, ax = plt.subplots(figsize=(9, 5), dpi=120)

# Histogramme (effectifs)
n, b, patches = ax.hist(
    x, bins=bins, density=False,
    edgecolor='white', linewidth=0.8, alpha=0.9
)

# Lignes moyenne / médiane
moy = np.mean(x)
med = np.median(x)
ax.axvline(moy, linestyle='--', linewidth=1.6, label=f"Moyenne = {moy:,.0f}".replace(",", " "))
ax.axvline(med, linestyle=':',  linewidth=1.6, label=f"Médiane = {med:,.0f}".replace(",", " "))

# Habillage
ax.set_title("Distribution de original_upb", pad=10)
ax.set_xlabel("original_upb")
ax.set_ylabel("Effectifs")
ax.grid(axis='y', linestyle=':', alpha=0.5)
ax.xaxis.set_major_formatter(fmt)
ax.yaxis.set_major_formatter(fmt)
ax.legend(frameon=False, loc='upper right')

# (Optionnel) Utile si très asymétrique :
# ax.set_xscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Exemple : df est votre DataFrame et la colonne s’appelle 'status'
# 1) Lister les modalités (valeurs distinctes)
df_train['default_24m'].unique()

# 2) Compter chaque modalité
df['status'].value_counts()

# 3) N’afficher que les lignes dont la modalité est 'default'
df_default = df[df['status'].eq('default')]        # équiv. : df.query("status == 'default'")

# 4) Nombre et proportion de 'default'
nb_default = df['status'].eq('default').sum()
prop_default = df['status'].eq('default').mean()   # proportion entre 0 et 1

# 5) Si la colonne est codée 0/1, la mapper vers des libellés puis (optionnel) en catégorie ordonnée
df['status'] = df['status'].map({1: 'default', 0: 'non-default'}).astype('category')

from pandas.api.types import CategoricalDtype
cat = CategoricalDtype(categories=['non-default', 'default'], ordered=True)
df['status'] = df['status'].astype(cat)

# 6) Afficher la catégorie 'default' (si dtype catégoriel) et vérifier qu’elle existe
'default' in df['status'].cat.categories
